In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important} <\style>"))
import matplotlib.pyplot as plt
%matplotlib inline

## working out costs

In [2]:
taker = 0.075/100
btc = 6720
margin = 0.075/100

fee = btc*taker*2
profit = btc*margin
fee,profit

(10.08, 5.04)

## function to work out contract specs

In [3]:
## change to the last friday of every month
from dateutil.relativedelta import relativedelta, FR

def contract_sym(date):
    contracts = [[datetime.datetime(date.year,3,1,12,0)+relativedelta(day=32)+relativedelta(weekday=FR(-1)),"H"],
                 [datetime.datetime(date.year,6,1,12,0)+relativedelta(day=32)+relativedelta(weekday=FR(-1)),"M"],
                 [datetime.datetime(date.year,9,1,12,0)+relativedelta(day=32)+relativedelta(weekday=FR(-1)),"U"],
                 [datetime.datetime(date.year,12,1,12,0)+relativedelta(day=32)+relativedelta(weekday=FR(-1)),"Z"],
                 [datetime.datetime(date.year+1,3,1,12,0)+relativedelta(day=32)+relativedelta(weekday=FR(-1)),"H"],
                 [datetime.datetime(date.year+1,6,1,12,0)+relativedelta(day=32)+relativedelta(weekday=FR(-1)),"M"]]
    
    for x in range(len(contracts)):
        if date <= contracts[x][0]:
            front_sym = contracts[x][1] + str(contracts[x][0].year)[2:4]
            second_sym = contracts[x+1][1] + str(contracts[x+1][0].year)[2:4]
            break
        else:
            continue
            
    return front_sym,second_sym 

#start = datetime.datetime(2018,3,28)
#front_sym,second_sym = contract_sym(start)
#print(front_sym,second_sym)

# Choose whether you want to compress tick or put into 1 minute below

## Input date you want to analyse here

# Compress and make into on minute!

In [8]:
start = datetime.datetime(2019,3,18)
end = datetime.datetime(2019,3,19) # datetime.datetime.now() #

file_start_name = start

master = pd.DataFrame()
while start < end:    
    print(start)
    start_str = str(start.strftime("%Y%m%d"))       
    try:
        quote_data = pd.read_csv(r"https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/quote/"+start_str+".csv.gz")
    except:
        print(start_str,"not available")
        start = start+datetime.timedelta(days=1)
        continue
        
    quote_data["timestamp"] = pd.to_datetime(quote_data["timestamp"],format='%Y-%m-%dD%H:%M:%S.%f000') 
    quote_data = quote_data[quote_data["symbol"]=="XBTUSD"]
    quote_data = quote_data[["timestamp","bidPrice","askPrice"]]
    quote_data.set_index("timestamp",inplace=True)
    quote_data = quote_data.dropna()
    quote_data = quote_data.resample("1min").ohlc()
    
    master = master.append(quote_data)

    #com_data.to_csv(r"C:\Users\PC\Documents\ArbAlgo\data\bitmex_hist_data"+"\\"+str(start.date())+"_combined_strip.csv")
    
    start = start+datetime.timedelta(days=1)
    print("done day")
print("FINISHED")

master = pd.DataFrame(master.to_records())
master.columns = ["timestamp","bid_open","bid_high","bid_low","bid_close","ask_open","ask_high","ask_low","ask_close"]
writer = pd.ExcelWriter(r"C:\Users\cc\Documents\market_data\\"+str(file_start_name.date())+"_to_"+str(end.date())+"_1min.xlsx")
master.to_excel(writer,"Sheet1")
writer.save()

2019-03-18 00:00:00
done day
FINISHED


In [ ]:
x = True

if not x:
    print("its not True")

### display candlesticks

In [ ]:
display(master.head())
master_data = master["bidPrice"]
master_data.loc[:,"timestamp"] = master_data.index
master_data = master_data[["timestamp","open","high","low","close"]]
master_data.head()

from highcharts import Highstock
from highcharts.highstock.highstock_helper import jsonp_loader
H = Highstock()

#data_url = 'http://www.highcharts.com/samples/data/jsonp.php?filename=aapl-ohlc.json&callback=?'
#data = jsonp_loader(data_url, sub_d = r'(\/\*.*\*\/)')
data = master_data.values.tolist()

H.add_data_set(data, 'ohlc', 'Price', dataGrouping = {
                            'units' : [[
                                'minute', # unit name
                                [1,5,15,60] # allowed multiples
                                        ]]})

##### options ####
options = {
    "chart": {
        "height":750,
        "width":1700
    },
    'rangeSelector' : {
        'selected' : 2
    },
    'title' : {
        'text' : 'Bitmex Perpetual'
    },
}

H.set_dict_options(options)

H